## Age Estimation Using Deep Learning
- https://www.pyimagesearch.com/2020/04/13/opencv-age-detection-with-deep-learning/
- Age and Gender Classification Using Convolutional Neural Networks, Levi & Hassner, 2015

In [14]:
import cv2
import os
import numpy as np

In [1]:
# Constant
AGE_BUCKETS = ['(0-2)', '(4-6)', '(8-12)', '(15-20)',
               '(25-32)', '(38-43)', '(48-53)', '(60-100)']

In [3]:
# Parameter (Models)
FACE_DETECTOR_FPATH = ['face_detector/deploy.prototxt',
                       'face_detector/res10_300x300_ssd_iter_140000.caffemodel']
AGE_ESTIMATOR_FPATH = ['age_estimator/age_deploy.prototxt',
                       'age_estimator/age_net.caffemodel']

In [11]:
# Parameter
CONF_TH = 0.5

In [20]:
# Face detection
def detect_face(model, frame, conf_th):
    # Preprocess
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104.0, 177.0, 123.0))

    # Inference
    model.setInput(blob)
    detections = model.forward()

    # loop over the detections
    h, w = frame.shape[:2]
    boxes = []
    for i in range(0, detections.shape[2]):
        # Confidence threshold
        if detections[0, 0, i, 2] > conf_th:
            # Limit bounding box coordinates
            detections[0, 0, i, 3:7] = np.maximum(detections[0, 0, i, 3:7], 0)
            detections[0, 0, i, 3:7] = np.minimum(detections[0, 0, i, 3:7], 1)
            
            # compute bounding box coordinates (x1, y1, x2, y2)
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            boxes.append(box.astype("int"))

    return boxes

In [17]:
# Age Estimation
def estimate_age(model, frame, boxes):
    # Process each bounding box
    ages = []
    for box in boxes:
        # Crop
        startX, startY, endX, endY = box
        face = frame[startY:endY, startX:endX, :]

        # Preprocess
        blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), (78.43, 87.77, 114.90))

        # Inference
        model.setInput(blob)
        preds = model.forward()

        # find age bucket with maximum probability
        i = preds[0].argmax()
        age = AGE_BUCKETS[i]
        ageConfidence = preds[0][i]
        ages.append((age, ageConfidence))

    return ages    

In [4]:
# Load Model
faceNet = cv2.dnn.readNet(FACE_DETECTOR_FPATH[0], FACE_DETECTOR_FPATH[1])
ageNet = cv2.dnn.readNet(AGE_ESTIMATOR_FPATH[0], AGE_ESTIMATOR_FPATH[1])

In [9]:
# Open Camera
cap = cv2.VideoCapture(0)

In [21]:
# Age estimation on video stream
cv2.namedWindow('Result', cv2.WINDOW_NORMAL)

while True:
    # Read frame
    ret, frame = cap.read()
    if not ret:
        break

    # Face detection (x1, y1, x2, y2)
    boxes = detect_face(faceNet, frame, CONF_TH)

    # Age estimation (Age Bucket, Prob.)
    ages = estimate_age(ageNet, frame, boxes)

    # Overlay
    for box, age in zip(boxes, ages):
        cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 0, 255), 1)
        cv2.putText(frame, '%s: %.2f' % (age[0], age[1]), (box[0], box[1]),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Show result
    cv2.imshow('Result', frame)
    if cv2.waitKey(1) == 27:
        break

cv2.destroyAllWindows()